# Framework for closure strategies

Execute the entire framework from the beginning simulating the clsoure stragegy on the road network.

In [ ]:
import sumolib
import pandas as pd
import numpy as np
from ast import literal_eval
import json
from utils import *
import shutil
from tqdm.auto import tqdm
import os
import glob
from pathlib import Path
import traci
import xml.etree.ElementTree

import warnings
warnings.filterwarnings("ignore")

#### parameters

In [ ]:
city='Milano_big'
k = '1'
exp_name = 'HE_top%s'%k

## ROAD NET
# road network path
road_network_path = "../data/road_net/%s/%s_road_network.net.xml"%(city,city)

# top road to remove csv
path_road_to_remove = '../data/simulations/%s/%s/%s_road_%s.csv'%(city,exp_name,city,exp_name)

# rand road to remove csv
path_rand_road_to_remove = '../data/simulations/%s/%s/rand/*.csv'%(city,exp_name)

# road_edge_mapping
path_road_edge_mapping = '../data/road_net/%s/%s_road_edge_map.csv'%(city,city)

# new road network path
road_network_removed_path = '../data/road_net/%s/%s_road_network_%s.net.xml'%(city,city,exp_name)

# new random road network prefix
rand_road_network_removed_prefix = '../data/road_net/%s/%s_road_network_rand%s'%(city,city,k)

# road net for sumocfg
road_network_sumocfg_path = '../data/simulations/%s_road_network.net.xml'%city


## MOBILITY DEMAND

# dict tile-edges path
dict_tile_edges_path = "../data/OD_matrices/%s_tile_edges_h3_8.json"%city

# dict mobility demand filename
dict_mobility_demand_path = "../data/mobility_demand/%s/Milano_big_MD_30k_vehicle_trip.json"%city

# .rou.xml mobility demand
mobility_demand_path = "../data/mobility_demand/%s/Milano_big_MD_30k.rou.xml"%city

# new dict mobility demand filename
new_dict_mobility_demand_path = "../data/mobility_demand/%s/Milano_big_MD_30k_vehicle_trip_%s.json"%(city,exp_name)

# new .rou.xml mobility demand
new_mobility_demand_path = "../data/mobility_demand/%s/Milano_big_MD_30k_%s.rou.xml"%(city,exp_name)

# new dict mobility demand prefix for random exp
new_dict_mobility_demand_rand_prefix = "../data/mobility_demand/%s/Milano_big_MD_30k_vehicle_trip_rand%s"%(city,k)

# new .rou.xml mobility demand prefix for random exp
new_mobility_demand_rand_prefix = "../data/mobility_demand/%s/Milano_big_MD_30k_rand%s"%(city,k)


## ROUTED PATHS
# folder + prefix of routed paths for the experiments
routed_path_filename_prefix = "../data/simulations/%s/%s/routed_paths/%s_TD_30k_%s"%(city,exp_name,city,exp_name)

# folder + prefix of routed paths for the experiments
rand_routed_path_filename_prefix = "../data/simulations/%s/%s/rand/routed_paths/%s_TD_30k_rand%s"%(city,exp_name,city,k)


## EXPERIMENTS
# path to folder containing the sumo simulation script
path_sumo_script = "../sumo_simulation_scripts/"

# folder containing the routed paths to simulate using SUMO
demand_folder_path = "../data/simulations/%s/%s/routed_paths/"%(city,exp_name)

# folder containing the routed paths to simulate using SUMO
rand_demand_folder_path = "../data/simulations/%s/%s/rand/routed_paths/"%(city,exp_name)

# where to save the simulation outputs
sumo_output_dir = "../data/simulations/%s/%s/sumo_out/"%(city,exp_name)

# where to save the simulation outputs
rand_sumo_output_dir = "../data/simulations/%s/%s/rand/sumo_out/"%(city,exp_name)

# the prefix of the folder resulting from a simulation
fold_prfx = exp_name
rand_fold_prfx = 'rand%s'%k



#### Setup directories

In [ ]:
Path('../data/simulations/%s/%s'%(city,exp_name)).mkdir(exist_ok=True)
Path('../data/simulations/%s/%s/routed_paths'%(city,exp_name)).mkdir(exist_ok=True)
Path('../data/simulations/%s/%s/sumo_out'%(city,exp_name)).mkdir(exist_ok=True)
Path('../data/simulations/%s/%s/plots'%(city,exp_name)).mkdir(exist_ok=True)
Path('../data/simulations/%s/%s/results'%(city,exp_name)).mkdir(exist_ok=True)
Path('../data/simulations/%s/%s/rand'%(city,exp_name)).mkdir(exist_ok=True)
Path('../data/simulations/%s/%s/rand/routed_paths'%(city,exp_name)).mkdir(exist_ok=True)
Path('../data/simulations/%s/%s/rand/sumo_out'%(city,exp_name)).mkdir(exist_ok=True)

## 1. Modify the road net

Create a new road network closing the road to be removed

In [ ]:
road_edge_map = pd.read_csv(path_road_edge_mapping)

In [ ]:
road_edge_tbr = pd.read_csv(path_road_to_remove)

In [ ]:
rand_road_edge_tbr_list = []
for file in sorted(glob.glob(path_rand_road_to_remove)):
    rand_road_edge_tbr = pd.read_csv(file)
    rand_road_edge_tbr_list.append(rand_road_edge_tbr)

In [ ]:
len_removed_roads = pd.merge(road_edge_tbr, road_edge_map, on=['road']).groupby(by=['road']).sum()
len_removed_roads_rands = []
for df in rand_road_edge_tbr_list:
    l = pd.merge(df, road_edge_map, on=['road']).groupby(by=['road']).sum()
    len_removed_roads_rands.append(l)

In [ ]:
edge_tbr = np.array([])

for edge_list in road_edge_tbr['edge_id']:
    edge_tbr = np.append(edge_tbr, literal_eval(edge_list))

In [ ]:
edge_tbr_rands = []

for df in rand_road_edge_tbr_list:
    rand_edge_tbr = np.array([])
    for edge_list in df['edge_id']:
        rand_edge_tbr = np.append(rand_edge_tbr, literal_eval(edge_list))
    edge_tbr_rands.append(rand_edge_tbr)

In [ ]:
#road_edge_tbr['road']

Print some statistics about the removed roads

In [ ]:
print('#road to remove: %s'%str(len(road_edge_tbr)))
print('#edges to remove: %s'%str(len(edge_tbr)))
print('Meters of removed roads: %s'%str(sum(len_removed_roads['edge_len'])))

In [ ]:
for i in range(len(rand_road_edge_tbr_list)):
    print('#road to remove %s: %s'%(str(i),str(len(rand_road_edge_tbr_list[i]))))
    #print('#edges to remove '+str(i)+': '+str(len(edge_tbr_rands[i])))
    #print('Meters of removed roads '+str(i)+': '+str(sum(len_removed_roads_rands[i]['edge_len'])))

In [ ]:
df_stats = pd.DataFrame({'exp': [exp_name],
                         'removed_road': [len(road_edge_tbr)],
                         'removed_edge': [len(edge_tbr)],
                         'removed_meters': [sum(len_removed_roads['edge_len'])]})

for i in range(len(rand_road_edge_tbr_list)):
    new_row = {'exp': 'rand'+k+'_'+str(i),
               'removed_road': len(rand_road_edge_tbr_list[i]),
               'removed_edge': len(edge_tbr_rands[i]),
               'removed_meters': sum(len_removed_roads_rands[i]['edge_len'])}
    df_stats = df_stats.append(new_row, ignore_index=True)

df_stats.to_csv('../data/simulations/'+city+'/'+exp_name+'/results/'+exp_name+'_stats.csv', index=False)

Create removed road net for top experiment

In [ ]:
def disallow_roads(road_net_path, removed_edges, road_net_removed_path):
    et = xml.etree.ElementTree.parse(road_net_path)
    root = et.getroot()
    for edge in removed_edges:
        tmp = root.find("./edge[@id='%s']"%edge)
        for lane in tmp.iter('lane'):
            if 'disallow' in lane.attrib:
                lane.attrib['disallow'] = '%s passenger'%lane.attrib['disallow']

    et.write(road_net_removed_path)

In [ ]:
%%time

disallow_roads(road_network_path, edge_tbr, road_network_removed_path)

Create removed road net for rand experiment

In [ ]:
pbar = tqdm(total=len(edge_tbr_rands))

for i in range(len(edge_tbr_rands)):

    disallow_roads(road_network_path, edge_tbr_rands[i], '%s_%s.net.xml'%(rand_road_network_removed_prefix,str(i)))

    pbar.update(1)
pbar.close()

## 2. Re-compute mobility demand

Load OD matrix and dict tile-edges

In [ ]:
with open(dict_tile_edges_path, 'r') as f:
    dict_tile_edges = json.load(f)

In [ ]:
dict_edge_tile = {}

for k, v in dict_tile_edges.items():
    for edge in v:
        dict_edge_tile[edge] = k

Load baseline Mobility Demand

In [ ]:
with open(dict_mobility_demand_path, 'r') as f:
    dict_mobility_demand = json.load(f)

In [ ]:
# Removed edges from each tile
#dict_tile_len = {}

#for k,v in dict_tile_edges.items():
#    len_before = len(v)
#    len_after = len([i for i in v if i not in edge_tbr])
#    dict_tile_len[k] = (len_before, len_after)    

In [ ]:
#dict_tile_len

In [ ]:
def recompute_traffic_from_matrix(dict_tile_edges, dict_edge_tile, edge_tbr, attr_from, to, road_network,
                               threshold_km=1, max_tries=100):
    
    
    # flags to control the change of from or to edge
    flag = False
    
    # cache to control if two tiles are no connected
    cache_tiles = []
    
    if attr_from in edge_tbr or to in edge_tbr:
        flag = True
    
    # edge_from and edge_to are not removed, check if exist a route with the new road_net
    if not flag:
        #if road_network.getOptimalPath(road_network.getEdge(attr_from),
        #                               road_network.getEdge(to),
        #                               vClass='passenger')[0] is not None:
        if has_valid_route_traci([attr_from, to]):
            return [attr_from, to]
        # from and to are not removed, but doesn't exist a path anymore
        else:
            flag = True
    
    # select the edges on the same tile of the previous removed edge    
    if flag:
        tile_from = dict_edge_tile[attr_from]
        edge_list_start = dict_tile_edges[tile_from]
        edge_list_start = [i for i in edge_list_start if i not in edge_tbr]
        # the tile was empty because of removed edges, pick a random one
        while len(edge_list_start) == 0:
            edge_list_start = dict_tile_edges[np.random.choice(list(dict_tile_edges.keys()))]
            edge_list_start = [i for i in edge_list_start if i not in edge_tbr]

        tile_to = dict_edge_tile[to]
        edge_list_end = dict_tile_edges[tile_to]
        edge_list_end = [i for i in edge_list_end if i not in edge_tbr]
        # the tile was empty because of removed edges, pick a random one
        while len(edge_list_end) == 0:
            edge_list_end = dict_tile_edges[np.random.choice(list(dict_tile_edges.keys()))]
            edge_list_end = [i for i in edge_list_end if i not in edge_tbr]

    tries = 0
    
    # code to select new edges from create_traffic_from_matrix in utils
    while tries < max_tries:
                
        ind_start = np.random.randint(0, len(edge_list_start))
        ind_end = np.random.randint(0, len(edge_list_end))

        edge_start = edge_list_start[ind_start]
        edge_end = edge_list_end[ind_end]

        # compute distance here
        lon_o, lat_o = gps_coordinate_of_edge_avg(road_network, edge_start)
        lon_d, lat_d = gps_coordinate_of_edge_avg(road_network, edge_end)
                
        d_km = distance_earth_km({"lat":lat_o, "lon":lon_o}, {"lat":lat_d, "lon":lon_d})
                
        if d_km >= threshold_km:
            #if road_network.getOptimalPath(road_network.getEdge(edge_start),
            #                               road_network.getEdge(edge_end),
            #                               vClass='passenger')[0] is not None:
            if has_valid_route_traci([edge_start, edge_end]):
                return [edge_start, edge_end]
            else:
                tries += 1
        else:
            tries+=1

        if tries == max_tries:
            
            # Repeat this until it finds a path, at each iteration change the the end tile
            while True:
                if (tile_from, tile_to) not in cache_tiles:
                    edge_list_start_P = np.random.permutation(edge_list_start)
                    edge_list_end_P = np.random.permutation(edge_list_end)

                    it=0

                    for edge_start, edge_end in ((e1, e2) for e1 in edge_list_start_P for e2 in edge_list_end_P):

                        lon_o, lat_o = gps_coordinate_of_edge_avg(road_network, edge_start)
                        lon_d, lat_d = gps_coordinate_of_edge_avg(road_network, edge_end)

                        d_km = distance_earth_km({"lat":lat_o, "lon":lon_o}, {"lat":lat_d, "lon":lon_d})

                        if d_km >= threshold_km:
                            #if road_network.getOptimalPath(road_network.getEdge(edge_start),
                            #                               road_network.getEdge(edge_end),
                            #                               vClass='passenger')[0] is not None:
                            if has_valid_route_traci([edge_start, edge_end]):
                                return [edge_start, edge_end]
                    cache_tiles.append((tile_from, tile_to))
                else:
                    # if two tiles are not connected yet, change the flow with a random one
                    tile_from = np.random.choice(list(dict_tile_edges.keys()))
                    edge_list_start = dict_tile_edges[tile_from]
                    edge_list_start = [i for i in edge_list_start if i not in edge_tbr]
                    tile_to = np.random.choice(list(dict_tile_edges.keys()))
                    edge_list_end = dict_tile_edges[tile_to]
                    edge_list_end = [i for i in edge_list_end if i not in edge_tbr]

In [ ]:
def recompute_traffic_from_matrix_v2(dict_tile_edges, dict_edge_tile, edge_tbr, attr_from, to, road_network,
                                     threshold_km=1, cache_tiles_validity={}, cache_path_validity={}):
    
    
    # flags to control the change of from or to edge
    flag = False
    
    
    if attr_from in edge_tbr or to in edge_tbr:
        flag = True
    
    # edge_from and edge_to are not removed, check if exist a route with the new road_net
    if not flag:
        if has_valid_route_traci([attr_from, to]):
            return [attr_from, to], cache_tiles_validity, cache_path_validity
        # from and to are not removed, but doesn't exist a path anymore
        else:
            flag = True
    
    # select the edges on the same tile of the previous removed edge    
    if flag:
        tile_from = dict_edge_tile[attr_from]
        edge_list_start = dict_tile_edges[tile_from]
        edge_list_start = [i for i in edge_list_start if i not in edge_tbr]
        # the tile was empty because of removed edges, pick a random one
        while len(edge_list_start) == 0:
            edge_list_start = dict_tile_edges[np.random.choice(list(dict_tile_edges.keys()))]
            edge_list_start = [i for i in edge_list_start if i not in edge_tbr]

        tile_to = dict_edge_tile[to]
        edge_list_end = dict_tile_edges[tile_to]
        edge_list_end = [i for i in edge_list_end if i not in edge_tbr]
        # the tile was empty because of removed edges, pick a random one
        while len(edge_list_end) == 0:
            edge_list_end = dict_tile_edges[np.random.choice(list(dict_tile_edges.keys()))]
            edge_list_end = [i for i in edge_list_end if i not in edge_tbr]

    
    # code to select new edges from create_traffic_from_matrix_v2 in utils
    # Repeat this until it finds a path, at each iteration change the the end tile
    while True:
        str_key = f"{tile_from}_AND_{tile_to}"
        if  str_key not in cache_tiles_validity:
            edge_list_start_P = np.random.permutation(edge_list_start)
            edge_list_end_P = np.random.permutation(edge_list_end)

            for edge_start, edge_end in ((e1, e2) for e1 in edge_list_start_P for e2 in edge_list_end_P):
                
                if has_valid_route_traci_v2([edge_start, edge_end], cache_path_validity, road_network, threshold_km):
                    return [edge_start, edge_end], cache_tiles_validity, cache_path_validity
            str_key = f"{tile_from}_AND_{tile_to}"        
            cache_tiles_validity[str_key] = False
        else:
            # if two tiles are not connected yet, change the flow with a random one
            tile_from = np.random.choice(list(dict_tile_edges.keys()))
            edge_list_start = dict_tile_edges[tile_from]
            edge_list_start = [i for i in edge_list_start if i not in edge_tbr]
            tile_to = np.random.choice(list(dict_tile_edges.keys()))
            edge_list_end = dict_tile_edges[tile_to]
            edge_list_end = [i for i in edge_list_end if i not in edge_tbr]

Compute mobility demand for top experiment

In [ ]:
# md_path: path of .rou.xml to modify
# new_md_path: path of the new .rou.xml
# roadnet_removed_path: path of the road network with removed roads
# sumocfg_path: sumocfg file used by traci
# dict_tile_edges: dictionary tile - edges
# dict_edge_tile: dictionary edge - tile
# edge_tbr: list of edges to be removed
# roadnet_baseline: path of the road network without removed roads
# dict_md: dictionary of original mobility demand
# new_md_json_path: path where to save the new dictionary of mobility demand

def recompute_mobility_demand(md_path, new_md_path, roadnet_removed_path, sumocfg_path, dict_tile_edges, dict_edge_tile, edge_tbr, roadnet_baseline, dict_md, new_md_json_path):

    changed_md = 0
    et = xml.etree.ElementTree.parse(md_path)
    root = et.getroot()
    road_network_removed = sumolib.net.readNet(roadnet_removed_path, withInternal=False)


    pbar = tqdm(total=len(root.findall('flow')))
    
    # copy of the network with removed edges for traci initialization
    shutil.copy(roadnet_removed_path, sumocfg_path)
    
    #init traci for fast route validation
    try:
        init_traci(warnings=False)
    except traci.TraCIException:
        pass
    
    cache_tiles={}
    cache_path={}
    
    for f in root.iter('flow'):
        
        new_edges, cache_tiles, cache_path = recompute_traffic_from_matrix_v2(dict_tile_edges, dict_edge_tile, edge_tbr, 
                                                                              f.attrib['from'], f.attrib['to'], road_network_removed,
                                                                              threshold_km = 0.8, cache_tiles_validity=cache_tiles,
                                                                              cache_path_validity=cache_path)
        
        if new_edges is None:
            print(f.attrib['from'], f.attrib['to'])
            print('No route available for an OD pair: check removed roads or pick low threshold_km')
            shutil.copy(roadnet_baseline, sumocfg_path)
            break
        else:
            if new_edges != [f.attrib['from'], f.attrib['to']]:
                changed_md +=1
                f.attrib['from'] = new_edges[0]
                f.attrib['to'] = new_edges[1]
                dict_md[f.attrib['id']]['edges'] = new_edges
        
        pbar.update(1)
    
    traci.close()
    
    # recopy of the baseline network traci initialization
    shutil.copy(roadnet_baseline, sumocfg_path)
    
    et.write(new_md_path)
    
    output_file = open(new_md_json_path, "w")
    json.dump(dict_md, output_file)
    output_file.close()
    
    pbar.close()
    
    print('Total mobility demand changed: '+str(changed_md))

Mobility demand for top experiment

In [ ]:
recompute_mobility_demand(mobility_demand_path,
                          new_mobility_demand_path,
                          road_network_removed_path,
                          road_network_sumocfg_path,
                          dict_tile_edges,
                          dict_edge_tile,
                          edge_tbr,
                          road_network_path, 
                          dict_mobility_demand,
                          new_dict_mobility_demand_path)

Compute mobility demand for random experiments

In [ ]:
pbar_main = tqdm(total=len(edge_tbr_rands))

for i in range(len(edge_tbr_rands)):
    
    recompute_mobility_demand(mobility_demand_path,
                              new_mobility_demand_rand_prefix+'_'+str(i)+'.rou.xml',
                              rand_road_network_removed_prefix+'_'+str(i)+'.net.xml',
                              road_network_sumocfg_path,
                              dict_tile_edges,
                              dict_edge_tile,
                              edge_tbr_rands[i],
                              road_network_path, 
                              dict_mobility_demand,
                              new_dict_mobility_demand_rand_prefix+'_'+str(i)+'.json')
    
    pbar_main.update(1)
    
pbar_main.close()

In [ ]:
## Check if two tiles are connected

#l_start = dict_tile_edges[dict_edge_tile['24545701']]
#l_end = dict_tile_edges[dict_edge_tile['25236612']]

#for edge_start, edge_end in ((e1, e2) for e1 in l_start for e2 in l_end):
#    r = road_network.getOptimalPath(road_network.getEdge('24545701'),
#                                            road_network.getEdge('25236612'),
#                                            vClass='passenger')
#    if r[0] is not None:
#        print(r[0])

## 3. Compute routed paths

In [ ]:
# number of traffic demand to generate
experiments = 3

Top experiment

In [ ]:
# output filename 
output_demand_filename_experiments = []
for e in range(experiments):
    output_demand_filename_experiments.append(routed_path_filename_prefix+'_'+str(e)+'.rou.xml')

In [ ]:
len(output_demand_filename_experiments)

Random experiment

In [ ]:
output_demand_filename_rand_experiments = []

for i in range(len(edge_tbr_rands)):
    out_demand_filename = []
    for e in range(experiments):
        out_demand_filename.append(rand_routed_path_filename_prefix+'_'+str(i)+'_'+str(e)+'.rou.xml')
    output_demand_filename_rand_experiments.append(out_demand_filename)

In [ ]:
len(np.array(output_demand_filename_rand_experiments).flatten())

#### Set duarouter's parameters

Top experiment

In [ ]:
weight = 7.5
rm_loops = "false"

seed_duarouter_list = []
seed_duarouter = None

if seed_duarouter is None:
    for e in range(experiments):
        seed_duarouter = np.random.randint(0, 9999999)
        seed_duarouter_list.append(seed_duarouter)
else:
    seed_duarouter_list = [seed_duarouter]*experiments

options_duarouter = "--weights.random-factor "+str(weight)+" --max-alternatives 10 --remove-loops "+rm_loops+" "\
    "--weights.interpolate true --weights.minor-penalty 0 "\
    " --routing-threads 8"

In [ ]:
len(seed_duarouter_list)

Random experiment

In [ ]:
seed_duarouter_rand_list = []

for i in range(len(edge_tbr_rands)):

    weight = 7.5
    rm_loops = "false"

    seed_duarouter_list = []
    seed_duarouter = None

    if seed_duarouter is None:
        for e in range(experiments):
            seed_duarouter = np.random.randint(0, 9999999)
            seed_duarouter_list.append(seed_duarouter)
    else:
        seed_duarouter_list = [seed_duarouter]*experiments
        
    seed_duarouter_rand_list.append(seed_duarouter_list)

    options_duarouter_rand = "--weights.random-factor "+str(weight)+" --max-alternatives 10 --remove-loops "+rm_loops+" "\
        "--weights.interpolate true --weights.minor-penalty 0 "\
        " --routing-threads 8"

In [ ]:
len(np.array(seed_duarouter_rand_list).flatten())

#### Apply duarouter

Execution time with experiments = 10 and n_vehicles=30k $\approx 10 min$

Top experiment

In [ ]:
for e in range(experiments):

    # prepare the command string for duarouter

    command_str = "duarouter --route-files "+new_mobility_demand_path+" "+\
            " --net-file "+road_network_removed_path+" "+options_duarouter+\
        " --random false --seed "+str(seed_duarouter_list[e])+\
        " --output-file "+output_demand_filename_experiments[e]

    # call duarouter process
    call_duarouter_command(command_str)

    # remove .alt file
    os.remove(output_demand_filename_experiments[e].split(".rou")[0]+".rou.alt"+output_demand_filename_experiments[e].split(".rou")[1])

    print("Created the routed demand [duarouter]: "+output_demand_filename_experiments[e])

Random experiment

In [ ]:
pbar = tqdm(total=len(edge_tbr_rands)*experiments)

for i in range(len(edge_tbr_rands)):

    for e in range(experiments):

        # prepare the command string for duarouter

        command_str = "duarouter --route-files "+new_mobility_demand_rand_prefix+'_'+str(i)+'.rou.xml'+" "+\
                " --net-file "+rand_road_network_removed_prefix+'_'+str(i)+'.net.xml'+" "+options_duarouter+\
            " --random false --seed "+str(seed_duarouter_rand_list[i][e])+\
            " --output-file "+output_demand_filename_rand_experiments[i][e]

        # call duarouter process
        call_duarouter_command(command_str)

        # remove .alt file
        os.remove(output_demand_filename_rand_experiments[i][e].split(".rou")[0]+".rou.alt"+output_demand_filename_rand_experiments[i][e].split(".rou")[1])

        print("Created the routed demand [duarouter]: "+output_demand_filename_rand_experiments[i][e])
        
        pbar.update(1)

## 4. SUMO experiments

The goal of the following notebook is to simulate, using SUMO, the vehicular traffic generated by the mixed Routed Paths to study the impact of different **percentages of routed vehicle** with respect to CO2 emissions.
___

We simulate the vehicular traffic generated by the routed paths using SUMO (Simulation of Urban MObility).
SUMO explicitly models each vehicle’s physics and dynamics, including their routes through the road network, allowing us to simulate vehicular traffic realistically, including traffic jams, queues at traffic lights, and slowdowns due to heavy traffic.
___

`run_sumo.py` may be executed with the following command:

`run_sumo.py net_path demand_path gui traj co2 ttime vpedge save_dir fold_prfx opt`

- `net_path`: the path of the SUMO road network;
- `demand_path`: the path of the mixed Routed Path to simulate;
- `gui`: whether to use the GUI (1) or not (0);
- `traj`: the collection mode for GPS traces: use real for all vehicles while none for no vehicles;
- `co2`: the collection mode for CO2 emissions at the edge level: use real for all vehicles while none for no vehicles;
- `ttime`: the collection mode for travel time: use real for all vehicles while none for no vehicles;
- `vpedge`: the collection mode for number of vehicle per edge: use real for all vehicles while none for no vehicles;
- `save_dir`: the path of the directory in which store the simulation results;
- `fold_prfx`: the prefix to use for the output directory;
- `opt`: the option with which to instantiate SUMO.

#### Set params

In [ ]:
# SUMO options
opt =  '"-W --ignore-junction-blocker 20 --time-to-impatience 30 --time-to-teleport 120 --scale 1"'

In [ ]:
gui = 0
max_hours = 4
# CO2, NOX, fuel consumed, travel time, speed of vehicles per edge
co2 = 'real'
nox = 'real'
fuel = 'real'
traveltime = 'real'
speed = 'real'
gps = 'none'
# vehicles per edge at each timestep
v_edge = 'real'
# veichles on the road at each timestep
v_step = 'real'

#### Load routed paths

In [ ]:
routed_paths = [demand_folder_path+f for f in os.listdir(demand_folder_path) if ".rou.xml" in f]

print("Routed Path to simulate: "+str(len(routed_paths)))

In [ ]:
rand_routed_paths = [rand_demand_folder_path+f for f in os.listdir(rand_demand_folder_path) if ".rou.xml" in f]

print("Random routed path to simulate: "+str(len(rand_routed_paths)))

#### Launch experiments

Execution time with experiments=10 and n_veichles=15k $\approx 1 hour$

Top experiment

In [ ]:
# traj co2 ttime vpedge

pbar = tqdm(total=len(routed_paths))

for demand_path in routed_paths:
    
    !cd {path_sumo_script} && python run_sumo.py -n {road_network_removed_path} -r {demand_path} -g {gui}\
    -s {sumo_output_dir} --max-hours {max_hours} --prefix {fold_prfx} --co2 {co2} --nox {nox}\
    --fuel {fuel} --traveltime {traveltime} --speed {speed} --gps {gps} --v-edge {v_edge} --v-step {v_step}\
    --sumo-opt {opt} >> foo0.txt
    pbar.update(1)
    print(demand_path)

Random experiment

In [ ]:
# traj co2 ttime vpedge

pbar = tqdm(total=len(rand_routed_paths))

for demand_path in rand_routed_paths:
    
    rand_exp_id = demand_path.split('/')[-1].split('_')[-2]
    road_net_removed_path = rand_road_network_removed_prefix+'_'+rand_exp_id+'.net.xml'
    
    !cd {path_sumo_script} && python run_sumo.py -n {road_net_removed_path} -r {demand_path} -g {gui}\
    -s {rand_sumo_output_dir} --max-hours {max_hours} --prefix {rand_fold_prfx+'_'+rand_exp_id} --co2 {co2} --nox {nox}\
    --fuel {fuel} --traveltime {traveltime} --speed {speed} --gps {gps} --v-edge {v_edge} --v-step {v_step}\
    --sumo-opt {opt} >> foo0.txt
    pbar.update(1)
    print(demand_path)